In [13]:
from insolAPI.WebAPI import API
import simplejson as json
import pandas as pd
import pendulum as pdl
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from datetime import datetime, timedelta
import warnings
from tqdm import tqdm
import sys
import pytz

In [2]:
time_args = dict(
    start=pdl.yesterday().subtract(months=1).to_datetime_string(),
    stop=(pdl.yesterday().add(days=1)).to_datetime_string(),
    timezone="UTC",
)

def read_json_config():
    """
    Read the config.json file and return the api key
    """
    with open("config.json") as f:
        config_data = json.load(f)

    installations = {}

    # Iterate through locations
    locations = config_data['locations']
    for location in locations:
        # Store details in a dictionary
        location_details = {
            'id': location['id'],
            'name': location['name'],
            'latitude': location['latitude'],
            'longitude': location['longitude'],
            'wind_threshold': location['wind_threshold'],
            'high_temperature_threshold': location['high_temperature_threshold'],
            'has_a_screen': location['has_a_screen'],
        }
        installations[location['name']] = location_details
    return installations, [config_data['api_key'], config_data['api_url']]


installation_path = "C:/Users/Insolight/Desktop/InsolReports/Scripts"
with open(installation_path + "/local.json") as f:
    local_data = json.load(f)
api = API(local_data["API_user"], local_data["API_pwd"], dev_space=False)

list_sensor = [
    "PAR",
    "IRRAD",
    "GII",
    "DNI",
    "DHI",
    "TEMP",
    "HUMI",
    "RAIN",
    "RAIN_TYPE",
    "RAIN_ACCUMULATED",
    "WIND",
    "WIND_DIR",
    "VIRTUAL",
    "LEAF_TEMP"
]

dict_instal = {
    "Etchelecu": 26,
    "Agroscope Series": 24,
    "Agroscope Berries": 25,
    "Bioschmid": 23,
}

In [19]:
sensor_number = 0
logs_joined = {}
dict_merged_df = {}
dict_df_missing_values = {}
df_log_unique_sensor = {}
print("Date range: " + time_args['start'] + " to " + time_args['stop'])
for instal in dict_instal:
    print(instal)
    sensor_number = 0
    logs_joined[instal] = {}
    dict_merged_df[instal] = {}
    dict_df_missing_values[instal] = {}
    df_log_unique_sensor[instal] = {}
    for sensor_type in tqdm(api.SensorsTypes):
        if str(sensor_type).split(".")[1] in list_sensor:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore", category=UserWarning)
                logs_joined[instal][list_sensor[sensor_number]] = api.get_sensor_channels_logs_joined(**time_args,sensor_type=sensor_type, install=dict_instal[instal])
                sensor_number += 1

    for sensor in list_sensor:
        if logs_joined[instal][sensor].empty:
            continue
        list_unique_sensors = logs_joined[instal][sensor]['sensor_name'].unique()
        for unique_sensor in list_unique_sensors:
            df_log_unique_sensor[instal][unique_sensor] = logs_joined[instal][sensor].loc[logs_joined[instal][sensor]['sensor_name']==unique_sensor].copy()
            df_log_unique_sensor[instal][unique_sensor].reset_index(inplace=True)
            df_log_unique_sensor[instal][unique_sensor].rename(columns={'index':'time'}, inplace=True)
            df_log_unique_sensor[instal][unique_sensor]['time'] = pd.to_datetime(df_log_unique_sensor[instal][unique_sensor]['time'])
            df_log_unique_sensor[instal][unique_sensor]['time'] = df_log_unique_sensor[instal][unique_sensor]['time'].dt.round('10min')
            df_log_unique_sensor[instal][unique_sensor].drop_duplicates(subset=['time','sensor_name'], inplace=True)
            #need to use the now end
            # date_range = pd.date_range(start=df_log_unique_sensor[instal][unique_sensor]['time'].min(), end=df_log_unique_sensor[instal][unique_sensor]['time'].max() + pd.DateOffset(0), freq='10min')
            date_range = pd.date_range(start=df_log_unique_sensor[instal][unique_sensor]['time'].min().tz_localize(None), end=pd.to_datetime(datetime.now().strftime("%Y-%m-%d %H:%M:%S")) + pd.DateOffset(0), freq='10min')
            date_range = date_range.tz_localize(pytz.timezone('UTC'))
            full_df = pd.DataFrame(date_range, columns=['time'])
            dict_merged_df[instal][unique_sensor] = pd.merge(full_df, df_log_unique_sensor[instal][unique_sensor], how='left', on="time")
            columns = dict_merged_df[instal][unique_sensor].columns
            dict_merged_df[instal][unique_sensor].rename(columns={columns[2]:'value_sensor'}, inplace=True)

            #counting missing values
            old_day = 0
            missing_values = 0
            dict_days = {}
            for index, row in dict_merged_df[instal][unique_sensor].iterrows():
                new_day = row['time'].day
                if new_day == old_day and pd.isnull(row["value_sensor"]):
                    missing_values += 1
                if new_day != old_day:
                    dict_days[datetime(year=row["time"].year, month=row["time"].month, day=row["time"].day) - timedelta(days=1)] = missing_values*10 / 1430 * 100
                    old_day = new_day
                    missing_values = 0
            dict_df_missing_values[instal][unique_sensor] = pd.DataFrame.from_dict(dict_days, orient='index')

    fig = go.Figure()
    for i in dict_df_missing_values[instal]:
        fig.add_trace(go.Scatter(x=dict_df_missing_values[instal][i].index, y=dict_df_missing_values[instal][i][0], name=i))
        fig.update_layout(yaxis_title='% of day disconnected', hovermode='x', title=instal, hoverlabel=dict(bgcolor="white"))
        #add a title

    fig.show()
    #save figure as html in a folder named "results"
    fig.write_html("../missing_sensors_recap/" + instal + '.html')

Date range: 2023-12-02 00:00:00 to 2024-01-03 00:00:00
Etchelecu


 20%|██        | 5/25 [00:05<00:21,  1.08s/it]


KeyboardInterrupt: 

# End of notebook

In [5]:
dict_df_per_day = {}
for instal in dict_instal:
    dict_df_per_day[instal] = pd.DataFrame()
    for i in dict_df_missing_values[instal]:
        # print(i)
        dict_df_per_day[instal][i] = dict_df_missing_values[instal][i][0]

In [10]:
dict_merged_df["Bioschmid"]["WIND02_EXT"]

,time,sensor_channel_id,value_sensor,integrated [° · h],gateway_id,local_channel_id,created_at,deleted_at,sensor_name,channel_name,...,serial_number,configuration_json,address,dx,dy,dz,nx,ny,zones_ids,zones_names
0,2023-11-19 00:00:00+00:00,1084.0,157.0,135.427078,28.0,86.0,2023-09-29T18:46:25.000000Z,None,WIND02_EXT,wind_direction,...,R23031310,[],86.0,0.0,0.0,0.0,1.0,1.0,"[1, 35, 36, 37]","[Steuerung Zone, Agrotrack, Agroverti, Kontrol..."
1,2023-11-19 00:10:00+00:00,1084.0,49.0,144.810410,28.0,86.0,2023-09-29T18:46:25.000000Z,None,WIND02_EXT,wind_direction,...,R23031310,[],86.0,0.0,0.0,0.0,1.0,1.0,"[1, 35, 36, 37]","[Steuerung Zone, Agrotrack, Agroverti, Kontrol..."
2,2023-11-19 00:20:00+00:00,1084.0,202.0,165.827087,28.0,86.0,2023-09-29T18:46:25.000000Z,None,WIND02_EXT,wind_direction,...,R23031310,[],86.0,0.0,0.0,0.0,1.0,1.0,"[1, 35, 36, 37]","[Steuerung Zone, Agrotrack, Agroverti, Kontrol..."
3,2023-11-19 00:30:00+00:00,1084.0,157.0,194.202087,28.0,86.0,2023-09-29T18:46:25.000000Z,None,WIND02_EXT,wind_direction,...,R23031310,[],86.0,0.0,0.0,0.0,1.0,1.0,"[1, 35, 36, 37]","[Steuerung Zone, Agrotrack, Agroverti, Kontrol..."
4,2023-11-19 00:40:00+00:00,1084.0,133.0,219.552078,28.0,86.0,2023-09-29T18:46:25.000000Z,None,WIND02_EXT,wind_direction,...,R23031310,[],86.0,0.0,0.0,0.0,1.0,1.0,"[1, 35, 36, 37]","[Steuerung Zone, Agrotrack, Agroverti, Kontrol..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4315,2023-12-18 23:10:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4316,2023-12-18 23:20:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4317,2023-12-18 23:30:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4318,2023-12-18 23:40:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df_log_unique_sensor["Bioschmid"]["WIND02_EXT"]

,time,sensor_channel_id,Wind direction [°],integrated [° · h],gateway_id,local_channel_id,created_at,deleted_at,sensor_name,channel_name,...,serial_number,configuration_json,address,dx,dy,dz,nx,ny,zones_ids,zones_names
0,2023-11-19 00:00:00+00:00,1084,157,135.427078,28,86,2023-09-29T18:46:25.000000Z,None,WIND02_EXT,wind_direction,...,R23031310,[],86,0,0,0,1,1,"[1, 35, 36, 37]","[Steuerung Zone, Agrotrack, Agroverti, Kontrol..."
5,2023-11-19 00:10:00+00:00,1084,49,144.810410,28,86,2023-09-29T18:46:25.000000Z,None,WIND02_EXT,wind_direction,...,R23031310,[],86,0,0,0,1,1,"[1, 35, 36, 37]","[Steuerung Zone, Agrotrack, Agroverti, Kontrol..."
12,2023-11-19 00:20:00+00:00,1084,202,165.827087,28,86,2023-09-29T18:46:25.000000Z,None,WIND02_EXT,wind_direction,...,R23031310,[],86,0,0,0,1,1,"[1, 35, 36, 37]","[Steuerung Zone, Agrotrack, Agroverti, Kontrol..."
19,2023-11-19 00:30:00+00:00,1084,157,194.202087,28,86,2023-09-29T18:46:25.000000Z,None,WIND02_EXT,wind_direction,...,R23031310,[],86,0,0,0,1,1,"[1, 35, 36, 37]","[Steuerung Zone, Agrotrack, Agroverti, Kontrol..."
25,2023-11-19 00:40:00+00:00,1084,133,219.552078,28,86,2023-09-29T18:46:25.000000Z,None,WIND02_EXT,wind_direction,...,R23031310,[],86,0,0,0,1,1,"[1, 35, 36, 37]","[Steuerung Zone, Agrotrack, Agroverti, Kontrol..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21013,2023-12-18 22:30:00+00:00,1084,163,3633.378418,28,86,2023-09-29T18:46:25.000000Z,None,WIND02_EXT,wind_direction,...,R23031310,[],86,0,0,0,1,1,"[1, 35, 36, 37]","[Steuerung Zone, Agrotrack, Agroverti, Kontrol..."
21015,2023-12-18 22:40:00+00:00,1084,203,3639.331055,28,86,2023-09-29T18:46:25.000000Z,None,WIND02_EXT,wind_direction,...,R23031310,[],86,0,0,0,1,1,"[1, 35, 36, 37]","[Steuerung Zone, Agrotrack, Agroverti, Kontrol..."
21016,2023-12-18 22:50:00+00:00,1084,127,3669.581055,28,86,2023-09-29T18:46:25.000000Z,None,WIND02_EXT,wind_direction,...,R23031310,[],86,0,0,0,1,1,"[1, 35, 36, 37]","[Steuerung Zone, Agrotrack, Agroverti, Kontrol..."
21018,2023-12-18 23:00:00+00:00,1084,104,0.388333,28,86,2023-09-29T18:46:25.000000Z,None,WIND02_EXT,wind_direction,...,R23031310,[],86,0,0,0,1,1,"[1, 35, 36, 37]","[Steuerung Zone, Agrotrack, Agroverti, Kontrol..."


# test

In [11]:
test = api.get_sensor_channels_logs_joined(**time_args,sensor_type=api.SensorsTypes.WIND, install=23)

c:\Users\Insolight\mambaforge\envs\insolinsights\Lib\site-packages\insolAPI\WebAPI.py:76: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.



In [12]:
test

,sensor_channel_id,Wind speed [m/s],integrated [m/s · h],gateway_id,local_channel_id,created_at,deleted_at,sensor_name,channel_name,sensor_type,...,serial_number,configuration_json,address,dx,dy,dz,nx,ny,zones_ids,zones_names
2023-11-19 00:00:44+00:00,1083,0.80,0.760278,28,85,2023-09-29T18:46:25.000000Z,None,WIND01_EXT,wind_speed,242,...,R23031202,[],71,0,0,0,1,1,"[1, 35, 36, 37]","[Steuerung Zone, Agrotrack, Agroverti, Kontrol..."
2023-11-19 00:00:45+00:00,1085,1.11,1.164896,28,87,2023-09-29T18:46:25.000000Z,None,WIND02_EXT,wind_speed,242,...,R23031310,[],86,0,0,0,1,1,"[1, 35, 36, 37]","[Steuerung Zone, Agrotrack, Agroverti, Kontrol..."
2023-11-19 00:01:44+00:00,1083,0.70,0.772778,28,85,2023-09-29T18:46:25.000000Z,None,WIND01_EXT,wind_speed,242,...,R23031202,[],71,0,0,0,1,1,"[1, 35, 36, 37]","[Steuerung Zone, Agrotrack, Agroverti, Kontrol..."
2023-11-19 00:01:45+00:00,1085,0.90,1.181646,28,87,2023-09-29T18:46:25.000000Z,None,WIND02_EXT,wind_speed,242,...,R23031310,[],86,0,0,0,1,1,"[1, 35, 36, 37]","[Steuerung Zone, Agrotrack, Agroverti, Kontrol..."
2023-11-19 00:02:44+00:00,1083,0.10,0.779444,28,85,2023-09-29T18:46:25.000000Z,None,WIND01_EXT,wind_speed,242,...,R23031202,[],71,0,0,0,1,1,"[1, 35, 36, 37]","[Steuerung Zone, Agrotrack, Agroverti, Kontrol..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-18 23:55:12+00:00,1083,0.60,0.323278,28,85,2023-09-29T18:46:25.000000Z,None,WIND01_EXT,wind_speed,242,...,R23031202,[],71,0,0,0,1,1,"[1, 35, 36, 37]","[Steuerung Zone, Agrotrack, Agroverti, Kontrol..."
2023-12-18 23:56:11+00:00,1083,1.00,0.336389,28,85,2023-09-29T18:46:25.000000Z,None,WIND01_EXT,wind_speed,242,...,R23031202,[],71,0,0,0,1,1,"[1, 35, 36, 37]","[Steuerung Zone, Agrotrack, Agroverti, Kontrol..."
2023-12-18 23:57:11+00:00,1083,0.60,0.349722,28,85,2023-09-29T18:46:25.000000Z,None,WIND01_EXT,wind_speed,242,...,R23031202,[],71,0,0,0,1,1,"[1, 35, 36, 37]","[Steuerung Zone, Agrotrack, Agroverti, Kontrol..."
2023-12-18 23:58:11+00:00,1083,1.00,0.363056,28,85,2023-09-29T18:46:25.000000Z,None,WIND01_EXT,wind_speed,242,...,R23031202,[],71,0,0,0,1,1,"[1, 35, 36, 37]","[Steuerung Zone, Agrotrack, Agroverti, Kontrol..."


In [13]:
df_log_unique_sensor["Bioschmid"]["WIND02_EXT"]

,time,sensor_channel_id,Wind direction [°],integrated [° · h],gateway_id,local_channel_id,created_at,deleted_at,sensor_name,channel_name,...,serial_number,configuration_json,address,dx,dy,dz,nx,ny,zones_ids,zones_names
0,2023-11-19 00:00:00+00:00,1084,157,135.427078,28,86,2023-09-29T18:46:25.000000Z,None,WIND02_EXT,wind_direction,...,R23031310,[],86,0,0,0,1,1,"[1, 35, 36, 37]","[Steuerung Zone, Agrotrack, Agroverti, Kontrol..."
5,2023-11-19 00:10:00+00:00,1084,49,144.810410,28,86,2023-09-29T18:46:25.000000Z,None,WIND02_EXT,wind_direction,...,R23031310,[],86,0,0,0,1,1,"[1, 35, 36, 37]","[Steuerung Zone, Agrotrack, Agroverti, Kontrol..."
12,2023-11-19 00:20:00+00:00,1084,202,165.827087,28,86,2023-09-29T18:46:25.000000Z,None,WIND02_EXT,wind_direction,...,R23031310,[],86,0,0,0,1,1,"[1, 35, 36, 37]","[Steuerung Zone, Agrotrack, Agroverti, Kontrol..."
19,2023-11-19 00:30:00+00:00,1084,157,194.202087,28,86,2023-09-29T18:46:25.000000Z,None,WIND02_EXT,wind_direction,...,R23031310,[],86,0,0,0,1,1,"[1, 35, 36, 37]","[Steuerung Zone, Agrotrack, Agroverti, Kontrol..."
25,2023-11-19 00:40:00+00:00,1084,133,219.552078,28,86,2023-09-29T18:46:25.000000Z,None,WIND02_EXT,wind_direction,...,R23031310,[],86,0,0,0,1,1,"[1, 35, 36, 37]","[Steuerung Zone, Agrotrack, Agroverti, Kontrol..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21013,2023-12-18 22:30:00+00:00,1084,163,3633.378418,28,86,2023-09-29T18:46:25.000000Z,None,WIND02_EXT,wind_direction,...,R23031310,[],86,0,0,0,1,1,"[1, 35, 36, 37]","[Steuerung Zone, Agrotrack, Agroverti, Kontrol..."
21015,2023-12-18 22:40:00+00:00,1084,203,3639.331055,28,86,2023-09-29T18:46:25.000000Z,None,WIND02_EXT,wind_direction,...,R23031310,[],86,0,0,0,1,1,"[1, 35, 36, 37]","[Steuerung Zone, Agrotrack, Agroverti, Kontrol..."
21016,2023-12-18 22:50:00+00:00,1084,127,3669.581055,28,86,2023-09-29T18:46:25.000000Z,None,WIND02_EXT,wind_direction,...,R23031310,[],86,0,0,0,1,1,"[1, 35, 36, 37]","[Steuerung Zone, Agrotrack, Agroverti, Kontrol..."
21018,2023-12-18 23:00:00+00:00,1084,104,0.388333,28,86,2023-09-29T18:46:25.000000Z,None,WIND02_EXT,wind_direction,...,R23031310,[],86,0,0,0,1,1,"[1, 35, 36, 37]","[Steuerung Zone, Agrotrack, Agroverti, Kontrol..."


In [14]:
old_day = 0
missing_values = 0
dict_test = {}
for index, row in dict_merged_df["Bioschmid"]["WIND02_EXT"].iterrows():
    new_day = row['time'].day
    if new_day == old_day and pd.isnull(row["value_sensor"]):
        missing_values += 1
    if new_day != old_day:
        # dict_days[datetime(year=row["time"].year, month=row["time"].month, day=row["time"].day) - timedelta(days=1)] = missing_values*10 / 1430 * 100
        dict_test[row["time"]] = missing_values
        old_day = new_day
        missing_values = 0

In [15]:
dict_test

{Timestamp('2023-11-19 00:00:00+0000', tz='UTC'): 0,
 Timestamp('2023-11-20 00:00:00+0000', tz='UTC'): 0,
 Timestamp('2023-11-21 00:00:00+0000', tz='UTC'): 0,
 Timestamp('2023-11-22 00:00:00+0000', tz='UTC'): 8,
 Timestamp('2023-11-23 00:00:00+0000', tz='UTC'): 0,
 Timestamp('2023-11-24 00:00:00+0000', tz='UTC'): 2,
 Timestamp('2023-11-25 00:00:00+0000', tz='UTC'): 8,
 Timestamp('2023-11-26 00:00:00+0000', tz='UTC'): 30,
 Timestamp('2023-11-27 00:00:00+0000', tz='UTC'): 21,
 Timestamp('2023-11-28 00:00:00+0000', tz='UTC'): 15,
 Timestamp('2023-11-29 00:00:00+0000', tz='UTC'): 5,
 Timestamp('2023-11-30 00:00:00+0000', tz='UTC'): 2,
 Timestamp('2023-12-01 00:00:00+0000', tz='UTC'): 19,
 Timestamp('2023-12-02 00:00:00+0000', tz='UTC'): 20,
 Timestamp('2023-12-03 00:00:00+0000', tz='UTC'): 112,
 Timestamp('2023-12-04 00:00:00+0000', tz='UTC'): 94,
 Timestamp('2023-12-05 00:00:00+0000', tz='UTC'): 95,
 Timestamp('2023-12-06 00:00:00+0000', tz='UTC'): 17,
 Timestamp('2023-12-07 00:00:00+0000

In [20]:
dict_df_missing_values["Etchelecu"]["WIND02_EXT"]

,0
2023-11-18,0.000000
2023-11-19,0.000000
2023-11-20,0.000000
2023-11-21,41.958042
2023-11-22,58.741259
2023-11-23,5.594406
2023-11-24,0.000000
2023-11-25,2.797203
2023-11-26,0.000000
2023-11-27,0.000000


In [9]:
date_range = pd.date_range(start=df_log_unique_sensor[instal][unique_sensor]['time'].min().tz_localize(None), end=pd.to_datetime(datetime.now().strftime("%Y-%m-%d %H:%M:%S")) + pd.DateOffset(0), freq='10min')

In [16]:
date_range = date_range.tz_localize(pytz.timezone('UTC'))

In [17]:
date_range

DatetimeIndex(['2023-12-05 08:10:00+00:00', '2023-12-05 08:20:00+00:00',
               '2023-12-05 08:30:00+00:00', '2023-12-05 08:40:00+00:00',
               '2023-12-05 08:50:00+00:00', '2023-12-05 09:00:00+00:00',
               '2023-12-05 09:10:00+00:00', '2023-12-05 09:20:00+00:00',
               '2023-12-05 09:30:00+00:00', '2023-12-05 09:40:00+00:00',
               ...
               '2024-01-03 08:50:00+00:00', '2024-01-03 09:00:00+00:00',
               '2024-01-03 09:10:00+00:00', '2024-01-03 09:20:00+00:00',
               '2024-01-03 09:30:00+00:00', '2024-01-03 09:40:00+00:00',
               '2024-01-03 09:50:00+00:00', '2024-01-03 10:00:00+00:00',
               '2024-01-03 10:10:00+00:00', '2024-01-03 10:20:00+00:00'],
              dtype='datetime64[ns, UTC]', length=4190, freq='10T')